# Volumen Lógico

La administración de volumenes crea una capa de abstraccion sobre el almacenamiento físico, lo que permite crear volumenes de almacenamiento lógico, lo que permite configuraciones más flexibles que usar el macenamiento físico directamente, además la configuración del almacenmiento del hardware está oculta del software por lo que puede cambiar su tamaño y moverse sin detener las aplicaciones ni desmontar los sistemas de archivos, esto puede reducir los costos operativos.


## Los volúmenes lógicos proporcionan las siguientes ventajas sobre el uso del almacenamiento físico directamente

- **Capacidad Flexible** Cuando se usan volúmenes lógicos los sistemas de archivos pueden extenderse a través de múltiples discos ya que puede agregar discos y particiones en un solo volumen logico

- **Resizable storage pools** Puede ampliar los volúmenes lógicos o reducir los volúmenes lógicos en tamaño con simples comandos de software, sin volver a formatear y volver a particionar los dispositivos de discos subyacentes

- **Reubicación de datos en línea** Para implementar subsistema de almacenamiento mas nuevos, más rápidos o más resistentes, puede mover los datos mientras su sistema está activo. Los datos pueden reorganizarse en discos mientras los discos están en uso. Por ejemplo, puede vaciar un disco intercambiable en caliente antes de extraerlo.

- **Conveniente nombre de dispositivos** Losv volumenes de almacenamiento lógico se pueden administrar en grupos con nombre presonalizados y definidos por el usuario

- **Disk striping** Puede crear un volumen lógico que distribuya los datos en uno o mas discos duros. Esto puede aumentar drásticamente el rendimiento

- **Volúmenes de espejo (Mirroring volumes)** Los volumenes lógicos proporcionan una forma conveniente de configurar un espejo para sus datos

- **Instantáneas de volumen** Al usar volumenes lógicos, puede tomar instantáneas del dispositivo para realizar copias de seguridad consistentes o para probar el efecto de los cambios sin afectar los datos reales

- **Thin volumes** Los volumenes lógicos pueden ser aprovisionados ligeramente. Esto le permite crear volumenes lógicos que son mas grandes que las extensiones disponibles.

- **Volúmenes de caché** Un volumen lógico de caché utiliza un pequeño volumen lógico que consiste en dispositivos de bloque rápido (como las unidades SSD) para mejorar el rendimiento de un volúmen lógico más grande y más lento al almacenar los bloques de uso frecuente en el volúmen lógico más pequeño y más rápido.


## Arquitectura LVM - Vista General


La unidad de almacenamiento físico subyacente de un volúmen lógico LVM es un dispositivo de bloque, como una partición o disco completo. Este dispositivo se incializa como un volúmen físico LVM (PV).

- **PV** es un Physical Volume
- **VG** es un Volume Group grupo de volúmenes físicos

Esto crea un espacio de almacenamiento a partir del cual se pueden asignar volúmenes lógicos (LVM), este proceso es análogo a la forma en la que los discos se dividen en pariciones. Los sistemas de archivos y las aplicaciones utilizan un volumen lógico, esto es lo más conveniente.

![01](03_img/01.png)


## Volúmenes Físicos

Las unidades de almacenamiento físico subyacente de un volúmen lógico LVM es un dispositivo de bloque, como una partición o un disco completo. Para usar el dispositivo de volúmen lógico LVM, el dospositivo debe inicializarse como un volúmen físico (PV). Al inicializar un dispositivo de bloque como volúmen físico, se coloca una etiqueta cerca del inicio del dispositivo. **La etiqueta LVM se coloca en el segundo sector de 512 bytes** pero puede sobreescribirse este valor por defecto para usar cualquiera de los primeros 4 sectores cuando se cree el volúmen físico esto permite que los usuarios de LVM coexistan con otros usuarios de otros sectores, si es necesario.

Una etiqueta LVM proporciona la información necesaria para la identificación correcta y el pedido de dispositivo para un dispositivo físico, ya que los dispositivos pueden aparecer en cualquier orden cuando se inicie el sistema, una etiqueta LVM permanece persistente en los reinicios y en un cluster. Esta  etiqueta identifica el dispositivo como un volumen físico LVM, contiene un identificador único aleatorio **UUID** para el volúmen físico, también alamcena el tamaño del sispositivo de bloque en byte y registra en dónde se almacenarán los metadatos LVM en el dispositivo.

Los metadatos LVM contienen la información de configuración de los grupos de volúmenes LVM en un sistema, de forma predeterminada, una copia identica de los metadatos se mantiene en cada área de metadatos en cada volumen físico de cada grupo de volúmenes. Los metadatos de LVM son pequeños y se almacenan como código **ASCII**. Actualmente, LVM también permite almacenar de 0, 1 o 2 copias idénticas de esos metadatos en cada volúmen físico, siendo el valor predeterminado 1, pero se puede cambiar al inicio de la configuración del LVM y posteriormente ya no podrá cambiar. La primera copia se almacena al inicio del dispositivo, poco después de la etiqueta, si hay una segunda copia, se coloca al final del dispositivo lo que permitirá recuperar los metadatos en caso de sobreescritura accidental

![02](03_img/02.png)


## Múltiples particiones en un disco

LVM permite crear volúmenes físicos a partir de particiones de discos RedHat recomienda que cree una sola partición que cubra todo el disco para etiquetar todo el disco como un volumen LVM por las siguientes razones:


### Conveniencia administrativa

Es más fácil hacer uns seguimiento del hardware en un sistema si cada disco real solo aparece una vez. Esto se vuelve particularmente cierto si falla un disco. Además, varios volúmenes físicos en un solo disco pueden provocar una advertencia del núcleo sobre tipos de partición desconocidos durante el inicio.

### Rendimiento de Striping

LVM no puede decir que dos volúmenes físicos estpan en el mismo disco físico. Si se crea un volúmen lógico seccionado cuando dos volúmenes físicos están en en el mismo disco físico, las bandas (stripes) podrían estar en diferentes particiones en el mismo disco. Esto daría lugar a una disminución de rendimiento en lugar de un aumento en el mismo.


## Grupos de volumen

Los volúmenes físicos se combinan en grupos de volúmenes (VG). Esto crea un conjunto de espacio en disco a partir del cuál se pueden asignar volúmenes lógicos

**Dentro de un grupo de volúmenes, el espacio en disco dispoible para la asignación se divide en unidades de un tamaño fijo llamadas extends (physical extends).** Una extensión es la unidad más pequeña de espacio que se puede asignar. Por ejemplo, si tenemos dos volúmenes físicos, y creamos el volumen de grupo, y dentro de los volúmenes físicos tenemos estos extends que fueron combinados para crear un volúmen de grupo y a partir de ese volúmen de grupo el volúmen lógico más pequeño que podemos crear es del tamaño de un extend y **el extend se puede definir y se puede cambiar**.

Dentro de un volumen físico las extensiones se denominan extensiones físicas o physical extends, un volúmen lógico se asigna en extensiones lógicas del mismo tamaño que las extensiones físicas, es decir, que si nosotros definimos o manejamos el valor por defecto, el tamaño de los logical extends de las extensiones lógicas es igual al physical extends.

El tamaño de la extensión es por lo tanto el mismo para todos los volúmenes lógicos en el grupo de volúmenes, el grupo de volúmenes asigna las extensiones lógicas en las extensiónes físicas. No podemos crear un volúmen de grupo con diferentes tamaños de physical extends, los extends siempre son iguales, pero si podemos crear un segundo, un tercero o un cuarto volumen de grupo con tamaño de extends diferentes.


## Volúmen lógico LVM

En LVM, un grupo de volúmenes se divide en volúmenes lógicos. Las siguientes secciones describen los diferentes tipos de volúmenes lógicos.


### Volúmenes lineales (Linear Volumes)

Un volúmen lineal agrega el espacio de uno o más volúmenes físicos en un volúmen lógico por ejemplo, **si se tienen dos discos de 60 GB, puede puede crear un volúmen lógico de 120GB**. El almacenamiento físico es concatenado.

La creación de un volumen lineal, asigna un rango de extensiones físicas a un area de un volumen lógico en orden

![03](03_img/03.png)

Desde el punto de vista de la aplicación, al final, hay un dispositivo con un tamaño final de 198 extends. Es importante aclarar que **los volúmenes físicos que conforman un volumen lógico no tienen que ser del mismo tamaño**

![04](03_img/04.png)

¿De cuantos megas es cada extend?

Se puede crear un volumen lineal de cualquier tamaño entre 1 y 300 extends. Tambien es posible configurar más de un volúmen lógico lineal del tamaño que se necesite del grupo de extensiones físicas

![05](03_img/05.png)


### Striped Logical Volumes

Cuando escribe datos en un volumen lógico LVM, el sistema de archivos distrinuye los datos a través de los volúmenes físicos subyacentes, Puede controlar la forma en que se escriben los datos en los volúmenes físicos creando un **volumen lógico seccionado**. Para lecturas y escrituras secuenciales de gran tamaño, esto puede mejorar la eficiencia de I/O de datos.

El striping mejora el rendimiento al escribir los datos en un numero predeterminado de volumenes físicos en forma de Round Robin con la creación de striped logical volumes los I/O se pueden hacer en paralelo, en algunas situaciones esto puede resultar en una ganancia de rendimiento casi lineal para cada volumne físico adicional en la banda o en el stripe.

![06](03_img/06.png)


En un volumen lógico seccionado el tamaño del stripe no puede exceder el tamaño de una extensión (extend), asi que si se usa de esta forma, vale la pena analizar el tamaño de el physical extend para nuestros volumenes lógicos.

Los volumenes lógicos segmentados de esta forma pueden extenderse concatenando otro conjunto de dispositivos al final del primer conjunto, sin embargo, para extender un volúmen lógico fragmentado, debe haber suficiente espacio libre en el conjunto de volumenes físicos subyacentes que conforman el grupo de volumenes para admitir la banda o el stripe.


### Volúmenes lógico RAID 

LVM admite RAID 0/1/4/5/6/10

Un volumen RAID LVM tiene muchas caracteristicas, pero destacaremos las principales:

- Los volúmenes lógicos RAID creados y administrados por medio de LVM aprovechan los controladores de kernel MD
- Las imágenes RAID1 pueden dividirse temporalmente en la matriz y fusionarse nuevamente en la matriz más adelante
- Los volúmenes RAID de LVM admiten instantáneas


### Volúmenes lógicos de aprovisionamiento thin (thin volumes)

Los volúmenes lógicos pueden ser aprovisionados ligeramente (thin). Esto le permite crear volumenes lógicos que son más grandes que las extensiones disponibles. Con el aprovisionamiento ligero (thin provisioning), puede administrar un grupo de almacenamiento de espacio libre, conocido como grupo delgado (thin pool), que se puede asignar a un número arbitrario de dispositivos cuando las aplicaciones lo requieren. Lo anterior, por ejemplo, quiere decir que el espacio de almacenamiento puede expadirse dinámicamente conforme la aplicación lo necesite. Al utilizar el almacenamiento ligero, un administrador de almacenamiento puede comprometer en exceso el almacenamiento físico, a menudo, evitando la necesidad de comprar almacenamiento adicional. Por ejemplo, si 10 clientes requieren cada uno 100 GB, el administrador de almacenamiento puede crear lo que parece ser un sistema de archivos de 100 GB para cada usuario, pero esta respaldado por menos almacenamiento real que se usa solo cuando es necesario. Cuando se utiliza este tipo de almacenamiento es importante que el administrador del almacenamiento supervise el grupo de almacenamiento y agregue más capacidad si comienza a llenarse, para lo cual hay herramientas de monitoreo. Para asegurarse de que se utilice todo el espacio de almacenamiento LVM admite el descarte de datos, esto permite la reutilización del espacio que antes era utilizado por un archivo descartado u otro rango de bloque. Ademas, este tipo de volumenes proporcionan soporte para una nueva implementación de instantáneas de copia de escritura **cow** que permite que los mismos datos compartan los mismos datos en la agrupación delgada.


### Volúmenes de Instantáneas (Snapshot Volumes)

La función de instantáneasde LVM brinda la capacidad de crear imágenes virtuales de un dispositivo en un instante particular sin causar una interrupción del servicio. Cuando se realiza un cambio en el dispositivo original (el de origen) después de tomar una instantánea, la función de instantánea hace una copia del área de datos modificada tal como era antes del cambio para que pueda reconstruir el estado del dispositivo.

Debido a que una instantánea copia solo las áreas de datos que cambian después de que se crea la instantánea, la función de instantánea requiere una cantidad mínima de almacenamiento, por ejemplo, con un origen que rara vez se actualiza, **De 3 a 5 porciento de la capacidad** del origen es suficiente para mantener la instantánea. El tama{o de una instantánea gobierna la capacidad de espacio para almacenar los cambios en el volumen de origen. 

Por ejemplo, una instantánea de corta duración, un periodo de tiempo corto de un volumen de lectura mayoritariamente como es el directorio **/usr** necesita menos espacio que una instantánea de larga duración de un volumen que ve un mayor numero de escrituras que tiene un mayor uso como es el caso del directorio **/home**. Si una instantánea se ejecuta completa, la instantánea debe ser válida ya que no puede rastrear los cambios en un volumen de origen, se debe controlar regularmente el volumen de origen, sin embargo, las instantáneas son completamente redimensionables por lo tanto si tiene la necesidad se puede incrementar el volumen de la instantánea para evitar que se caiga.

#### Hay varias aplicaciones para la función de instantaneas

- Normalmente, se toma una instantánea cuando se necesita realizar la copia de seguridad en un volúmen lógico sin detener el sistema en vivo que está actualizando continuamente los datos.

- Puede ejecutar el comando fsck en uns sistema de archivos de instantáneas para verificar la integridad del sistema de archivos y determinar si el sistema de archivos original requiere comprobación del sistema de archivos.

- Debido a que la instantánea es de lectura/escritura, puede probar que las aplicaciones contra los datos de producción tomando una instantánea y ejecutando pruebas contra la instantánea, sin tocar los datos reales.

- Puede crear volumenes LVM para usar con la virtualización. Las instantáneas de LVM se pueden usar en Red Hat para crear instantáneas de imágenes de invitados virtuales. Estas instantáneas pueden proporcionar una forma conveniente de modificar invitados existentes o crear nuevos invitados con un mínimo de almacenamiento adicional.


### Volúmenes de Instantáneas con aprovisionamiento fino Thinly-provisioned

Red Hat Enterprise Linux proporciona soporte para vol+umenes de instantáneas con aprovisionamiento fino. Los volúmenes de instantáneas delgadas permiten que muchos dispositivos virtuales se almacenen en el mismo volumen de datos. Esto simplifica la administración y permite el intercambio de datos entre volúmenes de instantáneas.

Beneficios:

- Un volumen thin de instantáneas puede reducir el uso del disco cuando hay varias instantáneas del mismo volumen de origen. 
- Si hay varias instantáneas del mismo origen, una escritura en el origen hará que una operación de COW conserve los datos. El aumento del número de instantáneas del origen no debería producir una desaceleración importante. 
- Los volúmenes de instantáneas finas se pueden utilizar como un origen de volumen lógico para otra instantánea. Esto permite una profundidad arbitraria de instantáneas recursivas (instantáneas de instantáneas de instantáneas ...). 
- Una instantánea de un volumen lógico thin también crea un volumen lógico thin. Esto no consume espacio de datos hasta que se requiera una operación de COW, o hasta que se escriba la instantánea. 
- No es necesario activar un volumen delgado de instantáneas con su origen, por lo que un usuario puede tener solo el origen activo mientras haya muchos volúmenes de instantáneas inactivas del origen. 


### Volúmenes de caché

LVM admite el uso de dispositivo de bloque rápido (como las unidades SSD) como cachés de escritura posterior o escritura directa para dispositivos de bloque más grandes y lentos. Los usuarios pueden crear volúmenes lógicos de caché para mejorar el rendimiento de sus volúmenes lógicos existentes o crear nuevos volúmenes lógicos de caché compuestos por un dispositivo pequeño y rápido junto con un dispositivo grande y lento.






